## Data sources

- HiC Data for Nui is here:
    - /input/genomic/plant/Vaccinium/corymbosum/AGRF_CAGRF21434_HJWHFDRXX


- 10X data for Nui and M7 here:
    - /input/genomic/plant/Vaccinium/corymbosum/AGRF_CAGRF18813_H7JY3DRXX


- ONT PromethION Nui (BB2020 and BB2020-2 are the same sample) here:
    - /input/genomic/plant/Vaccinium/corymbosum/Blueberry_PromethION_Apr2020


- ONT MinION Nui (BB2020) here:
    - /input/genomic/plant/Vaccinium/corymbosum/CAGRF21436/20200224_MinION/AGRF_CAGRFF21436_FAL87845_BB2020/


- 10X Supernova Assembly for 10X data here:
    - /output/genomic/plant/Vaccinium/corymbosum/2021_GenomeAssembly/Nui/01_Supernova

### Plan 
- base-calling for ONT samples using Guppy v5.
- Filter out MinION reads <1kb. Or higher...
- Cecilia has done the Supernova assembly for the 10X data.
- Use Flye to assemble ONT fastq
- Use quickmerge to merge the Supernova contigs + ONT contigs
- Use Salsa to improve assembly
- Tetraploid Haplotyping and gene annotation etc. 




In [7]:
WKDIR=/workspace/hraijc/BB_Nui_Assembly/blob
AL=/workspace/hraijc/BB_Nui_Assembly/Hybrid_assembly/Assembly_comparison/Nui_quickmerge2.fasta
ML=/workspace/hraijc/BB_Nui_Assembly/Hi-C_mapping/qm2_HiC_dedup.bam

In [8]:
mkdir ${WKDIR}
mkdir ${WKDIR}/log

mkdir: cannot create directory ‘/workspace/hraijc/BB_Nui_Assembly/blob’: File exists
mkdir: cannot create directory ‘/workspace/hraijc/BB_Nui_Assembly/blob/log’: File exists


: 1

In [9]:
cd ${WKDIR}

In [3]:
module load diamond

Loading diamond/0.9.24
  Loading requirement: singularity/3


In [4]:
bsub -J diamond2 -o $WKDIR/log/diamond2.out -e $WKDIR/log/diamond2.err -n 17 \
"diamond blastx \
--query ${AL} \
--max-target-seqs 1 \
--sensitive \
--threads 16 \
--db /workspace/hraijc/packages/diamond/uniprot_sprot.dmnd \
--evalue 1e-25 \
--outfmt 6 \
--out assembly.vs.uniprot_ref.mts1.1e25.out"

Job <636121> is submitted to default queue <lowpriority>.


In [17]:
cat assembly.vs.uniprot_ref.mts1.1e25.out | cut -f2 | cut -f2 -d "|" | sort| uniq > uniprot_ids.txt
#Then go to here to convert the UniProt IDs to NCBI TaxaIds:
# https://biodbnet.abcc.ncifcrf.gov/db/db2db.php
# save as tab deliniated UniProt01.txt

In [10]:
module load blobtools

Loading blobtools/1.0
  Loading requirement: pfr-python2/2.7.13


In [14]:
blobtools taxify -h

usage: blobtools taxify          -f FILE [-a INT] [-b INT] [-c INT]
                                    [-m FILE] [-s INT] [-t INT]
                                    [-i FILE] [-x INT] [-v FLOAT]
                                    [-o PREFIX] [-h|--help]

    Options:
        -h --help                           show this

    Options for similarity search input
        -f, --hit_file <FILE>               BLAST/Diamond similarity search result (TSV format).
                                                Defaults assume "-outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore'"
        -a, --hit_column_qseqid <INT>       Zero-based column of qseqid in similarity search result [default: 0]
                                                Change if different format than (-outfmt '6')
        -b, --hit_column_sseqid <INT>       Zero-based column of sseqid in similarity search result [default: 1]
                                                Chang

In [17]:
# taxify results
bsub -J blobtools -o $WKDIR/log/blobtools.out -e $WKDIR/log/blobtools.err -n 2 \
"blobtools taxify -f assembly.vs.uniprot_ref.mts1.1e25.out -m UniProt01.txt -s 0 -t 1"

Job <637660> is submitted to default queue <lowpriority>.


In [18]:
bsub -J blobtools2 -o $WKDIR/log/blobtools2.out -e $WKDIR/log/blobtools2.err -n 2 \
"blobtools create \
 -i $AL \
 -b $ML \
 -t assembly.vs.uniprot_ref.mts1.1e25.taxified.out \
 -o my_first_blobplot"

Job <637661> is submitted to default queue <lowpriority>.


In [4]:
#View results
bsub -J blobtools3 -o $WKDIR/log/blobtools3.out -e $WKDIR/log/blobtools3.err -n 2 \
"blobtools view -i my_first_blobplot.blobDB.json"

Job <636352> is submitted to default queue <lowpriority>.


In [19]:
bsub -J blobtools4 -o $WKDIR/log/blobtools4.out -e $WKDIR/log/blobtools4.err -n 2 \
"blobtools plot -i my_first_blobplot.blobDB.json"

Job <637665> is submitted to default queue <lowpriority>.
